In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2020-02-27 16:31:52--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M  19.5MB/s    in 7.8s    

2020-02-27 16:32:01 (19.5 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv                


In [62]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

KeyboardInterrupt: 

In [ ]:
# lets explore the datasets
movies_df.head()

In [ ]:
ratings_df.head()

In [ ]:
movies_df['year'] = movies_df['title'].str.extract(r'(\(\d\d\d\d\))',expand = False)
movies_df['year'] = movies_df['year'].str.extract(r'(\d\d\d\d)',expand = False)

In [ ]:
movies_df['title'] = movies_df['title'].str.replace(r'(\(\d\d\d\d\))','')

In [ ]:
movies_df['title'] = movies_df['title'].apply(lambda x : x.strip())

In [ ]:
movies_df.drop(columns = 'genres',inplace = True)

In [ ]:
movies_df

In [ ]:
ratings_df.drop(columns = 'timestamp',inplace = True)

In [ ]:
ratings_df.head()

In [ ]:
user_input = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
input_movies = pd.DataFrame(user_input)

In [18]:
#lets start building a recommendation engine
#now lets find movie id for this imput_movies table
input_id = movies_df[movies_df['title'].isin(input_movies['title'])]

In [24]:
input_movies = pd.merge(input_movies,input_id)

In [26]:
input_movies.drop(columns = 'year',inplace = True)

In [27]:
input_movies

,title,rating,movieId
0,"Breakfast Club, The",5.0,1968
1,Toy Story,3.5,1
2,Jumanji,2.0,2
3,Pulp Fiction,5.0,296
4,Akira,4.5,1274


In [63]:
user_subset = ratings_df[ratings_df['movieId'].isin(input_movies['movieId'])]
user_subset.head(2)

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0


In [64]:
user_subset_group = user_subset.groupby('userId')

In [65]:
user_subset_group = sorted(user_subset_group,key = lambda x : len(x[1]),reverse = True)

In [66]:
user_subset_group = user_subset_group[:100]

In [67]:
from scipy.stats import pearsonr
pearson_corr_dict = {}
import numpy as np
# we are sorting according to userId hence index in case of this iteration will be the userId
for name,group in user_subset_group:
    input_movies = input_movies.sort_values(by = 'movieId')
    group = group.sort_values(by = 'movieId')
    x = np.array(group['rating'])
    y = np.array(input_movies['rating'])
    pearson_corr_dict[name] = pearsonr(x,y)[0]
    
    
    
    


In [70]:
pearson_df = pd.DataFrame.from_dict(pearson_corr_dict,orient = 'index')

In [72]:
pearson_df.columns = ['similarity_index']

In [74]:
pearson_df['userId'] = pearson_df.index

In [76]:
pearson_df.reset_index(drop = True,inplace = True)

In [78]:
pearson_df = pearson_df.sort_values(by = 'similarity_index',ascending = False)

In [89]:
top_users = pearson_df[:50]
top_user

,similarity_index,userId
64,0.961678,12325
34,0.961538,6207
55,0.961538,10707
67,0.960769,13053
4,0.943456,1040
59,0.937614,11769
62,0.929294,12120
80,0.903584,15157
70,0.895144,13366
17,0.895144,3040


In [85]:
top_users_rating = top_users.merge(ratings_df,left_on = 'userId',right_on = 'userId',how = 'inner')

In [88]:
top_users_rating['weighted_ratings'] = top_users_rating['similarity_index'] * top_users_rating['rating']

In [95]:
top_users_rating = top_users_rating.groupby('movieId').sum()[['similarity_index','weighted_ratings']]

In [98]:
top_users_rating.columns = ['sum_similarity_index','sum_weighted_ratings']

In [101]:
recommendation_df = pd.DataFrame()
recommendation_df['weighted_average_recommendation_score'] = top_users_rating['sum_weighted_ratings'] / top_users_rating['sum_similarity_index']

In [103]:
recommendation_df['movie_id'] = recommendation_df.index

In [107]:
recommendation_df.sort_values(by = 'weighted_average_recommendation_score',ascending = False)

,weighted_average_recommendation_score,movie_id
movieId,,
26801,5.0,26801
6918,5.0,6918
91199,5.0,91199
1902,5.0,1902
6660,5.0,6660
...,...,...
27777,0.5,27777
5462,0.5,5462
5917,0.5,5917
